# Post Recommend notebook
This notebook is used to make readable recommendations in `txt` format for each algorithm. They will be saved in the following directory:
- `../../data/recommendations/city/year/output_algorithmName.txt`

In [1]:
import pandas as pd
import mysql.connector
import os
import sys
import pymysql

module_path = os.path.abspath(os.path.join('..'))

if module_path not in sys.path:
    python_path = os.path.join(module_path, 'python')
    sys.path.append(python_path)
    
from dusa_function_lib import build_directory_city_name, build_db_name

In [2]:
# mysql database connection
connection = pymysql.connect(
    host='localhost',
    user='eduardomv',
    password='*****',
    database='participatory_budgeting'
)
algorithms = ['rand', 'pop', 'pop_nc', 'ib', 'ub', 'mf', 'cb_cat', 'cb_loc', 'cbub_cat','cbub_loc', 'cbib_cat', 'cbib_loc']

cursor = connection.cursor()

cities = ['Cambridge', 'Miami', 'New York']
years = ['2014', '2015', '2016', '2017']

for city in cities:
    for year in years:
        directory_city_name = build_directory_city_name(city)
        
        for alg in algorithms:
            output_path = f"../../data/recommendations/{directory_city_name}/{year}/output_{alg}.txt"
            recs_path = f"../../data/recommendations/{directory_city_name}/{year}/rec_{alg}.csv"
            with open(output_path, 'w', encoding='utf-8') as f:
                f.write(f"{city} - {year} - {alg}\n")
                db_name = build_db_name(city, year)
                recommendations = pd.read_csv(recs_path, sep="|")
                for user_id in recommendations['userId'].unique():
                    # we obtain the items supported by this user from the database
                    query_ratings = f"""
                        SELECT itemId
                        FROM ratings
                        WHERE dataset = '{db_name}' AND userId = {user_id}
                    """
                    cursor.execute(query_ratings)
                    supported_items = cursor.fetchall()
                    supported_items = [item[0] for item in supported_items]
                    
                    if supported_items:
                        query_supported_items = f"""
                            SELECT id,title_gpt,description
                            FROM items
                            WHERE dataset = '{db_name}' AND id IN ({','.join(map(str, supported_items))})
                        """
                        cursor.execute(query_supported_items)
                        supported_items_detailed = cursor.fetchall()
                    else:
                        supported_items_detailed = []
                    
                    recommended_items = recommendations[recommendations['userId'] == user_id]['itemId'].tolist()[0:5]
                    
                    query_recommendations = f"""
                        SELECT id,title_gpt,description
                        FROM items
                        WHERE dataset = '{db_name}' AND id IN ({','.join(map(str, recommended_items))})
                    """
                    cursor.execute(query_recommendations)
                    recommended_items_detailed = cursor.fetchall()
                    
                    f.write(f"\nUser {user_id} - Supported Items:\n")
                    if len(supported_items_detailed) > 0:
                        for item in supported_items_detailed:
                            f.write(f"  - {item[0]}: {item[1]} - {item[2]}\n")
                    else:
                        f.write("  - No supported items\n")
                    
                    f.write(f"User {user_id} - Recommended Items ({alg}):\n")
                    for item in recommended_items_detailed:
                        f.write(f"  - {item[0]}: {item[1]} - {item[2]}\n")

cursor.close()
connection.close()